# Import libraries and access data

In [1]:
# Import libraries
from datetime import timedelta
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# comp90089-432714 Jiayuan
# comp90089-432102 Ron
# sturdy-web-432207-b2 Xinyi
# comp90089-432301 Molly
# Set up environment variables
project_id = 'comp90089-432714'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# Select features in mimic-iv dataset

In [2]:
icd = run_query("""
SELECT *
FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
WHERE (lower(long_title) LIKE '%cerebral infarction%' OR
       lower(long_title) LIKE '%stroke%' OR
       lower(long_title) LIKE '%cerebral hemorrhage%' OR
       lower(long_title) LIKE '%subarachnoid hemorrhage%' OR
       lower(long_title) LIKE '%intracranial hemorrhage%' OR
       lower(long_title) LIKE '%intracerebral hemorrhage%')
      AND lower(long_title) NOT LIKE '%without cerebral%'
      AND lower(long_title) NOT LIKE '%without stroke%'
      AND lower(long_title) NOT LIKE '%without infarction%'
      AND lower(long_title) NOT LIKE '%without hemorrhage%'
      AND lower(long_title) NOT LIKE '%without intracranial%'
      AND lower(long_title) NOT LIKE '%without intracerebral%'
      AND lower(long_title) NOT LIKE '%sunstroke%'
      AND lower(long_title) NOT LIKE '%heatstroke%'
      AND lower(long_title) NOT LIKE '%family%'
      AND lower(long_title) NOT LIKE '%history%'
      AND lower(long_title) NOT LIKE '%fetus%'
      AND lower(long_title) NOT LIKE '%newborn%'
      AND lower(long_title) NOT LIKE '%fracture%'
      AND lower(long_title) NOT LIKE '%following%'
      AND lower(long_title) NOT LIKE '%sequelae%'
      AND lower(long_title) NOT LIKE '%birth%'
      AND lower(long_title) NOT LIKE '%not resulting in%'
      AND lower(long_title) NOT LIKE '%nihss%'
      AND (lower(long_title) NOT LIKE '%traumatic%' OR lower(long_title) LIKE '%nontraumatic%')
""")
len(icd)
icd


<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,icd_code,icd_version,long_title
0,34660,9,Persistent migraine aura with cerebral infarct...
1,34661,9,Persistent migraine aura with cerebral infarct...
2,34662,9,Persistent migraine aura with cerebral infarct...
3,34663,9,Persistent migraine aura with cerebral infarct...
4,430,9,Subarachnoid hemorrhage
...,...,...,...
185,I636,10,Cerebral infarction due to cerebral venous thr...
186,I638,10,Other cerebral infarction
187,I6381,10,Other cerebral infarction due to occlusion or ...
188,I6389,10,Other cerebral infarction


In [3]:
icd.to_csv('icd.csv')

In [4]:
icd_codes = list(icd['icd_code'])
icd_codes.append('I64')

icd_codes_str = ', '.join([f"'{code}'" for code in icd_codes])

In [5]:
# select patient information
df1_demographics = run_query(f"""
WITH stroke_patients AS (
    SELECT
        p.subject_id,
        a.hadm_id,
        p.gender,
        p.anchor_age,
        a.race,
        a.admittime,
        a.dischtime,
        a.hospital_expire_flag,
        a.admission_type,
        a.insurance,
        a.marital_status,
        a.race
    FROM
        `physionet-data.mimiciv_hosp.patients` p
    JOIN
        `physionet-data.mimiciv_hosp.admissions` a ON p.subject_id = a.subject_id
    JOIN
        `physionet-data.mimiciv_hosp.diagnoses_icd` d ON a.hadm_id = d.hadm_id
    WHERE
        d.icd_code IN ({icd_codes_str})
)
SELECT * FROM stroke_patients
""")
# count the df1_demographics
# df1_demographics.head(1000)

# 'I64', 'G464', 'G463', 'I694', 'I619', 'I639'

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [6]:
# select ICD codes
df3_diagnoses = run_query(f"""
WITH stroke_diagnoses AS (
    SELECT
        d.subject_id,
        d.hadm_id,
        d.icd_code,
        d.icd_version
    FROM
        `physionet-data.mimiciv_hosp.diagnoses_icd` d
    WHERE
        d.icd_code IN ({icd_codes_str})
)
SELECT * FROM stroke_diagnoses
""")
# df3_diagnoses.head(20)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [7]:
# select labevents
df5_lab_events = run_query(f"""
WITH stroke_lab_results AS (
    SELECT
        lab.subject_id,
        lab.hadm_id,
        lab.itemid,
        lab.valuenum,
        lab.valueuom,
        lab.charttime
    FROM
        `physionet-data.mimiciv_hosp.labevents` lab
    JOIN
        `physionet-data.mimiciv_hosp.diagnoses_icd` d ON lab.hadm_id = d.hadm_id
    WHERE
        d.icd_code IN ({icd_codes_str})
)
SELECT * FROM stroke_lab_results
""")
# df5_lab_events.head(1000000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [8]:
# select intime and outtime in icu
df6_icu_stays = run_query(f"""
WITH stroke_icu_stays AS (
    SELECT
        icu.subject_id,
        d.hadm_id,
        icu.stay_id,
        icu.intime,
        icu.outtime,
        icu.los
    FROM
        `physionet-data.mimiciv_icu.icustays` icu
    JOIN
        `physionet-data.mimiciv_hosp.diagnoses_icd` d ON icu.hadm_id = d.hadm_id
    WHERE
        d.icd_code IN ({icd_codes_str})
)
SELECT * FROM stroke_icu_stays
""")
# df6_icu_stays.head(1000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [9]:
# select discharge information
df7_discharge = run_query(f"""
WITH stroke_discharge AS (
    SELECT
        p.subject_id,
        a.hadm_id,
        a.dischtime,
        a.deathtime,
        a.discharge_location
    FROM
        `physionet-data.mimiciv_hosp.patients` p
    JOIN
        `physionet-data.mimiciv_hosp.admissions` a
    ON
        p.subject_id = a.subject_id
    JOIN
        `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON
        a.hadm_id = d.hadm_id
    JOIN
        `physionet-data.mimiciv_hosp.d_icd_diagnoses` icd
    ON
        d.icd_code = icd.icd_code
    WHERE
        icd.icd_code IN ({icd_codes_str})
        )
SELECT * FROM stroke_discharge
""")
# df7_discharge.head(1000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [10]:
# select procedures information
df8_procedures = run_query(f"""
WITH stroke_procedures AS (
    SELECT
        subject_id,
        stay_id,
        itemid,
        starttime,
        endtime
    FROM
        `physionet-data.mimiciv_icu.procedureevents`
    WHERE
        stay_id IN (
            SELECT stay_id
            FROM `physionet-data.mimiciv_icu.icustays`
            WHERE subject_id IN (
                SELECT DISTINCT subject_id
                FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
                JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` icd
                ON d.icd_code = icd.icd_code
            WHERE
                d.icd_code IN ({icd_codes_str})
                )
            )
    )
    SELECT * FROM stroke_procedures
""")
# df8_procedures.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [11]:
# select prescriptions about stroke
df9_prescriptions = run_query(f"""
WITH stroke_prescriptions AS (
    SELECT
        p.subject_id,
        p.hadm_id,
        p.drug,
        p.formulary_drug_cd,
        p.starttime,
        p.stoptime,
        p.dose_val_rx,
        p.dose_unit_rx
    FROM
        `physionet-data.mimiciv_hosp.prescriptions` p
    JOIN
        `physionet-data.mimiciv_hosp.diagnoses_icd` d ON p.hadm_id = d.hadm_id
    WHERE
        d.icd_code IN ({icd_codes_str})
)
SELECT * FROM stroke_prescriptions
""")
# df9_prescriptions.head(1000000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [12]:
# select medical history and diagnosis
df10_medication = run_query(f"""
WITH stroke_medication AS (
    SELECT
        p.subject_id,
        d.icd_code,
        icd.long_title AS diagnosis_description,
        pr.drug AS medication_name
    FROM
        `physionet-data.mimiciv_hosp.patients` p
    JOIN
        `physionet-data.mimiciv_hosp.admissions` a
    ON
        p.subject_id = a.subject_id
    JOIN
        `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON
        a.hadm_id = d.hadm_id
    JOIN
        `physionet-data.mimiciv_hosp.d_icd_diagnoses` icd
    ON
        d.icd_code = icd.icd_code
    LEFT JOIN
        `physionet-data.mimiciv_hosp.prescriptions` pr
    ON
        a.hadm_id = pr.hadm_id
    WHERE
        d.icd_code IN ({icd_codes_str})
)
SELECT * FROM stroke_medication
""")
# df10_medication.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [13]:
# select blood gas analysis data
df11_blood_gas_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT subject_id, hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    WHERE d.icd_code IN ({icd_codes_str})
),
blood_gas_analysis AS (
    SELECT
        bg.subject_id,
        bg.hadm_id,
        bg.charttime,
        bg.ph,
        bg.po2 AS PaO2,
        bg.pco2 AS PaCO2
    FROM
        `physionet-data.mimiciv_derived.bg` bg
    WHERE
        bg.hadm_id IN (
            SELECT DISTINCT hadm_id
            FROM stroke_patients
        )
        AND (bg.ph IS NOT NULL OR
             bg.po2 IS NOT NULL OR
             bg.pco2 IS NOT NULL)
)
SELECT *
FROM blood_gas_analysis
""")
# df11_blood_gas_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [14]:
# select chemistry analysis data
df12_chemistry_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT subject_id, hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    WHERE d.icd_code IN ({icd_codes_str})
),
chemistry_analysis AS (
    SELECT
        c.subject_id,
        c.hadm_id,
        c.charttime,
        c.sodium,
        c.potassium,
        c.calcium,
        c.creatinine,
        c.bun
    FROM
        `physionet-data.mimiciv_derived.chemistry` c
    WHERE
        c.hadm_id IN (
            SELECT DISTINCT hadm_id
            FROM stroke_patients
        )
        AND (c.sodium IS NOT NULL OR
             c.potassium IS NOT NULL OR
             c.calcium IS NOT NULL OR
             c.creatinine IS NOT NULL OR
             c.bun IS NOT NULL)
)
SELECT *
FROM chemistry_analysis
""")
# df12_chemistry_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [15]:
# select urine data
df13_urine_output_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
),
urine_output_analysis AS (
    SELECT
        u.stay_id,
        u.charttime,
        u.urineoutput AS urine_output_total
    FROM
        `physionet-data.mimiciv_derived.urine_output` u
    WHERE
        u.stay_id IN (
            SELECT DISTINCT stay_id
            FROM stroke_patients
        )
        AND u.urineoutput IS NOT NULL
)
SELECT *
FROM urine_output_analysis
""")
# df13_urine_output_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [16]:
# select coagulation analysis data
df14_coagulation_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT subject_id, hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    WHERE d.icd_code IN ({icd_codes_str})
),
coagulation_analysis AS (
    SELECT
        c.subject_id,
        c.hadm_id,
        c.charttime,
        c.pt,
        c.inr,
        c.ptt
    FROM
        `physionet-data.mimiciv_derived.coagulation` c
    WHERE
        c.hadm_id IN (
            SELECT DISTINCT hadm_id
            FROM stroke_patients
        )
        AND (c.pt IS NOT NULL OR
             c.inr IS NOT NULL OR
             c.ptt IS NOT NULL)
)
SELECT *
FROM coagulation_analysis
""")
# df14_coagulation_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [17]:
# select vital signs analysis data
df15_vital_signs_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
),
vital_signs_analysis AS (
    SELECT
        vs.stay_id,
        vs.charttime,
        vs.sbp AS systolic_bp,
        vs.dbp AS diastolic_bp,
        vs.mbp AS mean_arterial_pressure,
        vs.heart_rate,
        vs.resp_rate,
        vs.temperature,
        vs.spo2,
        vs.glucose
    FROM
        `physionet-data.mimiciv_derived.vitalsign` vs
    WHERE
        vs.stay_id IN (
            SELECT DISTINCT stay_id
            FROM stroke_patients
        )
    ORDER BY
        vs.charttime
)
SELECT *
FROM vital_signs_analysis
""")
# df15_vital_signs_analysis.head(1000000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [18]:
# select vasoactive agents analysis data
df16_vasoactive_agents_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
),
vasoactive_agents_analysis AS (
    SELECT
        v.stay_id,
        v.starttime,
        v.endtime,
        v.norepinephrine,
        v.dopamine,
        v.epinephrine,
        v.phenylephrine,
        v.vasopressin,
        v.dobutamine,
        v.milrinone
    FROM
        `physionet-data.mimiciv_derived.vasoactive_agent` v
    WHERE
        v.stay_id IN (
            SELECT DISTINCT stay_id
            FROM stroke_patients
        )
    AND (v.norepinephrine IS NOT NULL OR
         v.dopamine IS NOT NULL OR
         v.epinephrine IS NOT NULL OR
         v.phenylephrine IS NOT NULL OR
         v.vasopressin IS NOT NULL OR
         v.dobutamine IS NOT NULL OR
         v.milrinone IS NOT NULL)
)
SELECT *
FROM vasoactive_agents_analysis
""")
# df16_vasoactive_agents_analysis.head(100000)


<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [19]:
# select crrt analysis data
df17_crrt_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
),
crrt_analysis AS (
    SELECT
        c.stay_id,
        c.charttime,
        c.crrt_mode,
        c.access_pressure,
        c.blood_flow,
        c.dialysate_rate,
        c.hourly_patient_fluid_removal,
        c.ultrafiltrate_output
    FROM
        `physionet-data.mimiciv_derived.crrt` c
    WHERE
        c.stay_id IN (
            SELECT DISTINCT stay_id
            FROM stroke_patients
        )
    AND (c.crrt_mode IS NOT NULL OR
         c.access_pressure IS NOT NULL OR
         c.blood_flow IS NOT NULL OR
         c.dialysate_rate IS NOT NULL OR
         c.hourly_patient_fluid_removal IS NOT NULL OR
         c.ultrafiltrate_output IS NOT NULL)
)
SELECT *
FROM crrt_analysis
""")
# df17_crrt_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [20]:
# select ventilation analysis data
df18_ventilation_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
),
ventilation_analysis AS (
    SELECT
        v.stay_id,
        v.starttime,
        v.endtime,
        v.ventilation_status
    FROM
        `physionet-data.mimiciv_derived.ventilation` v
    WHERE
        v.stay_id IN (
            SELECT DISTINCT stay_id
            FROM stroke_patients
        )
    AND v.ventilation_status IS NOT NULL
)
SELECT *
FROM ventilation_analysis
""")
# df18_ventilation_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [21]:
# select kdigo analysis data
df19_kdigo_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
),
kdigo_analysis AS (
    SELECT
        k.subject_id,
        k.stay_id,
        k.charttime,
        k.aki_stage_creat AS kdigo_stage,
        k.creat AS creatinine_change,
        k.uo_rt_6hr,
        k.uo_rt_12hr,
        k.uo_rt_24hr,
        k.aki_stage_uo,
        k.aki_stage_crrt
    FROM
        `physionet-data.mimiciv_derived.kdigo_stages` k
    WHERE
        k.stay_id IN (
            SELECT DISTINCT stay_id
            FROM stroke_patients
        )
    AND (k.aki_stage_creat IS NOT NULL OR
         k.creat IS NOT NULL OR
         k.uo_rt_6hr IS NOT NULL OR
         k.uo_rt_12hr IS NOT NULL OR
         k.uo_rt_24hr IS NOT NULL OR
         k.aki_stage_uo IS NOT NULL OR
         k.aki_stage_crrt IS NOT NULL)
)
SELECT *
FROM kdigo_analysis
""")
# df19_kdigo_analysis.head(100000)

<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [22]:
# select LODS & OASIS score
df20_combined_score_analysis = run_query(f"""
WITH stroke_patients AS (
    SELECT DISTINCT ie.stay_id
    FROM `physionet-data.mimiciv_icu.icustays` ie
    JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` d
    ON ie.hadm_id = d.hadm_id
    WHERE d.icd_code IN ({icd_codes_str})
)
SELECT
    sp.stay_id,
    l.subject_id AS lods_subject_id,
    l.lods AS lods_score,
    o.subject_id AS oasis_subject_id,
    o.oasis AS oasis_score
FROM stroke_patients sp
LEFT JOIN `physionet-data.mimiciv_derived.lods` l
    ON sp.stay_id = l.stay_id AND l.lods IS NOT NULL
LEFT JOIN `physionet-data.mimiciv_derived.oasis` o
    ON sp.stay_id = o.stay_id AND o.oasis IS NOT NULL
LIMIT 100000;
""")
# df20_combined_score_analysis.head(100000)


<ipython-input-1-18b3c9b7b350>:31: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


# Data merged

In [23]:
# all patients informations including intime and outtime of icu

# step1: merged df1_demographics and df6_icu_stays
df_merged1 = df1_demographics.merge(df6_icu_stays, on=['subject_id', 'hadm_id'], how='left')

# step2: merged df_merged and df7_discharge
df_merged1 = df_merged1.merge(df7_discharge, on=['subject_id', 'hadm_id'], how='left')

# drop 'race_1' and dischtime_y' columns
df_merged1 = df_merged1.drop(columns=['race_1', 'dischtime_y'])

# check df_merged1
df_merged1.head(5)

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,marital_status,stay_id,intime,outtime,los,deathtime,discharge_location
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,0,EW EMER.,Medicare,MARRIED,<NA>,NaT,NaT,NaN,NaT,HOME
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,MARRIED,32626706,2183-12-06 09:49:01,2183-12-08 18:24:12,2.357766,NaT,SKILLED NURSING FACILITY
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,MARRIED,32626706,2183-12-06 09:49:01,2183-12-08 18:24:12,2.357766,NaT,SKILLED NURSING FACILITY
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,MARRIED,32626706,2183-12-06 09:49:01,2183-12-08 18:24:12,2.357766,NaT,SKILLED NURSING FACILITY
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,MARRIED,32626706,2183-12-06 09:49:01,2183-12-08 18:24:12,2.357766,NaT,SKILLED NURSING FACILITY


In [24]:
# check the count of the same item_id in df5_lab_events
df5_lab_events.groupby('itemid').size().sort_values(ascending=False).head(5)

,0
itemid,
50983,159406
50971,154685
50902,151833
50912,144405
51006,143843


Urea Nitrogen (BUN): 51006\
Hematocrit: 51221\
Hemoglobin: 51222\
Platelet Count: 51265\
White Blood Cells (WBC): 51301\
INR (PT): 51237\
Prothrombin Time (PT): 51274\
Partial Thromboplastin Time (PTT): 51275\
Alkaline Phosphatase: 50863\
Aspartate Aminotransferase (AST): 50878\
Bilirubin, Total: 50885\


In [25]:
# Specified list of itemids
target_itemids = [51006, 51221, 51222, 51265, 51301, 51237, 51274, 51275, 50863, 50878, 50885]

# Filter data for the specified itemids
filtered_df = df5_lab_events[df5_lab_events['itemid'].isin(target_itemids)]

# Sort by subject_id, itemid, and charttime
filtered_df = filtered_df.sort_values(by=['subject_id', 'itemid', 'charttime'])

# Remove duplicate tests, keeping only the first test result
first_lab_results = filtered_df.drop_duplicates(subset=['subject_id', 'itemid'], keep='first')

# View the first few rows
first_lab_results.head(10000)

# Create separate tables for each itemid
for itemid in target_itemids:
    globals()[f'item_{itemid}'] = df5_lab_events[df5_lab_events['itemid'] == itemid].sort_values(by=['subject_id', 'charttime']).drop_duplicates(subset=['subject_id'], keep='first')

# Print each generated table
target_itemids = [51006, 51221, 51222, 51265, 51301, 51237, 51274, 51275, 50863, 50878, 50885]

In [26]:
# Create a dictionary to store each itemid's table name and corresponding DataFrame value
item_tables = {
    "item_51006": item_51006,
    "item_51221": item_51221,
    "item_51222": item_51222,
    "item_51265": item_51265,
    "item_51301": item_51301,
    "item_51237": item_51237,
    "item_51274": item_51274,
    "item_51275": item_51275,
    "item_50863": item_50863,
    "item_50878": item_50878,
    "item_50885": item_50885
}

# Use df_merged1 as the base dataset
df_merged2 = df_merged1.copy()

# Iterate through each table and merge it into df_merged2
for table_name, df_item in item_tables.items():
    # Extract only the columns subject_id, hadm_id, and valuenum
    df_item_subset = df_item[['subject_id', 'hadm_id', 'valuenum']].copy()

    # Rename the valuenum column to the table_name
    df_item_subset.rename(columns={'valuenum': table_name}, inplace=True)

    # Merge the subset with the base dataframe
    df_merged2 = df_merged2.merge(df_item_subset, on=['subject_id', 'hadm_id'], how='left')

# View the data
df_merged2.head(5)

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,item_51221,item_51222,item_51265,item_51301,item_51237,item_51274,item_51275,item_50863,item_50878,item_50885
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,0,EW EMER.,Medicare,...,35.7,12.4,234.0,6.8,NaN,NaN,NaN,NaN,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Calculate drug usage-related features based on each patient's hospital admission ID and patient ID
df_prescription_features = df9_prescriptions.groupby(['subject_id', 'hadm_id']).agg(
    total_drugs=('drug', 'count'),  # Total number of drug usage
    unique_drugs=('drug', 'nunique'),  # Number of unique drug types used
    total_duration=('starttime', lambda x: (df9_prescriptions['stoptime'] - df9_prescriptions['starttime']).dt.total_seconds().sum() / (60 * 60 * 24)),  # Total duration of drug usage (in days)
    total_dose=('dose_val_rx', 'sum'),  # Total dose
).reset_index()

# View data
df_prescription_features.head(5)

,subject_id,hadm_id,total_drugs,unique_drugs,total_duration,total_dose
0,10001877,25679292,18,15,3.437168e+06,50510012.53510502.50.52011000.510020
1,10002155,28439444,14,12,3.437168e+06,200.512.50.4320.4141750.5100075014
2,10003299,21404960,13,13,3.437168e+06,011580310512.510.581500075
3,10003299,21743184,12,11,3.437168e+06,0.5405002001015000325-6503100325
4,10003299,22087674,7,7,3.437168e+06,325325-6500.5301001


In [28]:
# Select the required columns
df_prescription_subset = df_prescription_features[['subject_id', 'hadm_id', 'total_drugs', 'unique_drugs']]

# Merge df_merged2 and df_prescription_subset
df_merged3 = df_merged2.merge(df_prescription_subset, on=['subject_id', 'hadm_id'], how='left')

# View data
df_merged3.head(5)

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,item_51265,item_51301,item_51237,item_51274,item_51275,item_50863,item_50878,item_50885,total_drugs,unique_drugs
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,0,EW EMER.,Medicare,...,234.0,6.8,NaN,NaN,NaN,NaN,NaN,NaN,19.0,13.0
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0


In [29]:
# Select the required columns
df_score_subset = df20_combined_score_analysis[['lods_subject_id', 'stay_id', 'lods_score', 'oasis_score']]

# Rename the column 'lods_subject_id' to 'subject_id' for merging with df_merged3
df_score_subset.rename(columns={'lods_subject_id': 'subject_id'}, inplace=True)

# Merge df_merged3 and df_score_subset
df_merged4 = df_merged3.merge(df_score_subset, on=['subject_id', 'stay_id'], how='left')

# View data
df_merged4.head(5)

<ipython-input-29-8da19e8f2f71>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score_subset.rename(columns={'lods_subject_id': 'subject_id'}, inplace=True)


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,item_51237,item_51274,item_51275,item_50863,item_50878,item_50885,total_drugs,unique_drugs,lods_score,oasis_score
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,0,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,19.0,13.0,<NA>,<NA>
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0,6,28
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0,6,28
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0,6,28
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,296.0,71.0,6,28


In [30]:
# checj the count of same subject_id in df8_procedures
df8_procedures.groupby('itemid').size().sort_values(ascending=False).head(100)

,0
itemid,
224275,15818
225459,11982
224277,9683
221214,7835
225752,5795
...,...
227551,30
225447,29
229366,27


In [31]:
# Calculate the occurrence count for each itemid
grouped_data = df8_procedures.groupby('itemid').size()

# Filter out itemids with counts greater than 99
filtered_data = grouped_data[grouped_data > 99].sort_values(ascending=False)

# view data
filtered_data

,0
itemid,
224275,15818
225459,11982
224277,9683
221214,7835
225752,5795
...,...
227719,112
229532,109
229526,109


In [32]:
# Specified list of surgical itemids
# target_procedures = [224275, 225459, 224277, 221214, 229351, 225402, 225792, 229581]
target_procedures = set(grouped_data[grouped_data > 99].index)

# Filter the data for the specified itemids
filtered_procedures = df8_procedures[df8_procedures['itemid'].isin(target_procedures)]

# Count the number of surgeries each patient has for different surgical itemids
procedure_counts = filtered_procedures.groupby(['subject_id', 'itemid']).size().unstack(fill_value=0)

# Rename columns
procedure_counts.columns = [f'procedure_{int(col)}' for col in procedure_counts.columns]

# Merge the count results into df_merged4
df_merged5 = df_merged4.merge(procedure_counts, on='subject_id', how='left')

# View data
df_merged5.head(5)

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,procedure_229298,procedure_229351,procedure_229380,procedure_229519,procedure_229526,procedure_229532,procedure_229580,procedure_229581,procedure_229582,procedure_229614
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,0,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,0,ELECTIVE,Medicare,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Filter itemids with surgical procedure counts greater than 99
target_procedures = set(grouped_data[grouped_data > 99].index)

# Filter the data for the specified itemids
filtered_procedures = df8_procedures[df8_procedures['itemid'].isin(target_procedures)]

# Count the number of surgeries each patient has for different surgical itemids
procedure_counts = filtered_procedures.groupby(['subject_id', 'itemid']).size().unstack(fill_value=0)

# Rename columns to the format of procedure_itemid
procedure_counts.columns = [f'procedure_{int(col)}' for col in procedure_counts.columns]

# Merge the count results with the df_merged4 dataframe based on subject_id
df_merged5 = df_merged4.merge(procedure_counts, on='subject_id', how='left')

# Replace values of 0 with NaN
df_merged5.replace(0, np.nan, inplace=True)

# View the first 100 rows of the merged dataframe to ensure correct merging
print(df_merged5.head(5))

# View data
df_merged5.head(5)

   subject_id   hadm_id gender  anchor_age   race           admittime  \
0    19596808  26675460      M          79  WHITE 2128-06-04 16:13:00   
1    15378103  21479747      M          86  WHITE 2183-11-22 13:39:00   
2    15378103  21479747      M          86  WHITE 2183-11-22 13:39:00   
3    15378103  21479747      M          86  WHITE 2183-11-22 13:39:00   
4    15378103  21479747      M          86  WHITE 2183-11-22 13:39:00   

          dischtime_x  hospital_expire_flag admission_type insurance  ...  \
0 2128-06-05 11:36:00                  <NA>       EW EMER.  Medicare  ...   
1 2183-12-11 15:59:00                  <NA>       ELECTIVE  Medicare  ...   
2 2183-12-11 15:59:00                  <NA>       ELECTIVE  Medicare  ...   
3 2183-12-11 15:59:00                  <NA>       ELECTIVE  Medicare  ...   
4 2183-12-11 15:59:00                  <NA>       ELECTIVE  Medicare  ...   

  procedure_229298  procedure_229351 procedure_229380 procedure_229519  \
0              NaN      

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,procedure_229298,procedure_229351,procedure_229380,procedure_229519,procedure_229526,procedure_229532,procedure_229580,procedure_229581,procedure_229582,procedure_229614
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Step 1: Find the first blood gas analysis measurement for each patient
first_blood_gas_measurement = df11_blood_gas_analysis.groupby(['subject_id', 'hadm_id']).apply(
    lambda x: x.sort_values('charttime').iloc[0]
).reset_index(drop=True)

# Step 2: Keep only the required columns (ph, PaO2, PaCO2)
first_blood_gas_measurement = first_blood_gas_measurement[['subject_id', 'hadm_id', 'ph', 'PaO2', 'PaCO2']]

# Step 3: Merge the first blood gas measurement data with df_merged5
df_merged6 = df_merged5.merge(first_blood_gas_measurement, on=['subject_id', 'hadm_id'], how='left')

# Step 4: View data
df_merged6.head(5)

<ipython-input-34-ce065e3902d2>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_blood_gas_measurement = df11_blood_gas_analysis.groupby(['subject_id', 'hadm_id']).apply(


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,procedure_229519,procedure_229526,procedure_229532,procedure_229580,procedure_229581,procedure_229582,procedure_229614,ph,PaO2,PaCO2
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.45,457.0,34.0
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.45,457.0,34.0
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.45,457.0,34.0
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.45,457.0,34.0


In [35]:
# Step 1: Find the first electrolyte and renal function measurement for each patient
first_chemistry_measurement = df12_chemistry_analysis.groupby(['subject_id', 'hadm_id']).apply(
    lambda x: x.sort_values('charttime').iloc[0]
).reset_index(drop=True)

# Step 2: Keep only the required columns (sodium, potassium, calcium, creatinine, bun)
first_chemistry_measurement = first_chemistry_measurement[['subject_id', 'hadm_id', 'sodium', 'potassium', 'calcium', 'creatinine', 'bun']]

# Step 3: Merge the first electrolyte and renal function measurement data with df_merged6
df_merged7 = df_merged6.merge(first_chemistry_measurement, on=['subject_id', 'hadm_id'], how='left')

# Step 4: View data
df_merged7.head(5)

<ipython-input-35-dae6df2ff811>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_chemistry_measurement = df12_chemistry_analysis.groupby(['subject_id', 'hadm_id']).apply(


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,procedure_229582,procedure_229614,ph,PaO2,PaCO2,sodium,potassium,calcium,creatinine,bun
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,139.0,4.0,8.3,0.8,12.0
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0


In [36]:
# Step 1: Find the first urine output measurement for each patient
first_urine_output = df13_urine_output_analysis.groupby('stay_id').apply(
    lambda x: x.sort_values('charttime').iloc[0]
).reset_index(drop=True)

# Step 2: Keep only the required columns (stay_id, urine_output_total)
first_urine_output = first_urine_output[['stay_id', 'urine_output_total']]

# Step 3: Map stay_id to subject_id and hadm_id for merging
# Assume df_merged7 contains stay_id linkage information
df_merged8 = df_merged7.merge(first_urine_output, on='stay_id', how='left')

# Step 4: View data
df_merged8.head(5)

<ipython-input-36-5e636994bafa>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_urine_output = df13_urine_output_analysis.groupby('stay_id').apply(


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,procedure_229614,ph,PaO2,PaCO2,sodium,potassium,calcium,creatinine,bun,urine_output_total
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,139.0,4.0,8.3,0.8,12.0,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0,400.0
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0,400.0
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0,400.0
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,NaN,7.45,457.0,34.0,138.0,4.4,8.0,1.5,43.0,400.0


In [37]:
# Step 1: Find the first coagulation function measurement for each patient
first_coagulation_measurement = df14_coagulation_analysis.groupby(['subject_id', 'hadm_id']).apply(
    lambda x: x.sort_values('charttime').iloc[0]
).reset_index(drop=True)

# Step 2: Keep only the required columns (pt, inr, ptt)
first_coagulation_measurement = first_coagulation_measurement[['subject_id', 'hadm_id', 'pt', 'inr', 'ptt']]

# Step 3: Merge the first coagulation function measurement data with df_merged8
df_merged9 = df_merged8.merge(first_coagulation_measurement, on=['subject_id', 'hadm_id'], how='left')

# Step 4: View data
df_merged9.head(5)

<ipython-input-37-be439f462be5>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_coagulation_measurement = df14_coagulation_analysis.groupby(['subject_id', 'hadm_id']).apply(


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,PaCO2,sodium,potassium,calcium,creatinine,bun,urine_output_total,pt,inr,ptt
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,139.0,4.0,8.3,0.8,12.0,NaN,NaN,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,34.0,138.0,4.4,8.0,1.5,43.0,400.0,18.8,1.8,28.3
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,34.0,138.0,4.4,8.0,1.5,43.0,400.0,18.8,1.8,28.3
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,34.0,138.0,4.4,8.0,1.5,43.0,400.0,18.8,1.8,28.3
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,34.0,138.0,4.4,8.0,1.5,43.0,400.0,18.8,1.8,28.3


In [38]:
# Step 1: Find the first non-null value for each vital sign parameter
def get_first_valid_vital_signs(group):
    first_valid = {}

    # For each vital sign parameter, sort by charttime and get the first non-null value
    for col in ['systolic_bp', 'diastolic_bp', 'mean_arterial_pressure', 'heart_rate', 'resp_rate', 'temperature', 'spo2', 'glucose']:
        non_null_data = group.dropna(subset=[col]).sort_values('charttime')
        if not non_null_data.empty:
            first_valid[col] = non_null_data.iloc[0][col]
        else:
            first_valid[col] = None  # If there is no valid data, fill with None

    # Get the corresponding stay_id
    first_valid['stay_id'] = group['stay_id'].iloc[0]

    return pd.Series(first_valid)

# Apply the function and drop all records where the return values are empty
first_vital_signs = df15_vital_signs_analysis.groupby('stay_id').apply(get_first_valid_vital_signs).reset_index(drop=True)

# Step 2: Keep only the required columns (This step can be omitted since Step 1 already returns the needed columns)
# first_vital_signs = first_vital_signs[['stay_id', 'systolic_bp', 'diastolic_bp', 'mean_arterial_pressure', 'heart_rate', 'resp_rate', 'temperature', 'spo2', 'glucose']]

# Step 3: Merge the first vital sign data with df_merged9
df_merged10 = df_merged9.merge(first_vital_signs, on='stay_id', how='left')

# Step 4: View data
df_merged10.head(5)

<ipython-input-38-4e1e11969b46>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_vital_signs = df15_vital_signs_analysis.groupby('stay_id').apply(get_first_valid_vital_signs).reset_index(drop=True)


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,inr,ptt,systolic_bp,diastolic_bp,mean_arterial_pressure,heart_rate,resp_rate,temperature,spo2,glucose
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,1.8,28.3,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,1.8,28.3,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,1.8,28.3,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,1.8,28.3,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0


In [39]:
# Step 1: Find the first mechanical ventilation record for each patient
first_ventilation_data = df18_ventilation_analysis.groupby('stay_id').apply(
    lambda x: x.sort_values('starttime').iloc[0]  # Sort by starttime and take the first record
).reset_index(drop=True)

# Step 2: Keep only the starttime and endtime columns
first_ventilation_data = first_ventilation_data[['stay_id', 'starttime', 'endtime']]

# Step 3: Merge the first ventilation status data with df_merged10 to create df_merged11
df_merged11 = df_merged10.merge(first_ventilation_data, on='stay_id', how='left')

# Step 4: View data
df_merged11.head(5)

<ipython-input-39-ab892529da6c>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_ventilation_data = df18_ventilation_analysis.groupby('stay_id').apply(


,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,systolic_bp,diastolic_bp,mean_arterial_pressure,heart_rate,resp_rate,temperature,spo2,glucose,starttime,endtime
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,169.0,80.0,117.0,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00


In [40]:
# Calculate hospital stay duration (discharge time - admission time) (Unit: days)
df_merged11['hospital_stay_duration'] = (df_merged11['dischtime_x'] - df_merged11['admittime']).dt.total_seconds() / (60 * 60 * 24)

# Calculate ICU stay duration (Unit: days)
df_merged11['icu_stay_duration'] = (df_merged11['outtime'] - df_merged11['intime']).dt.total_seconds() / (60 * 60 * 24)

# Calculate ventilation duration (Unit: days)
df_merged11['ventilation_duration'] = (df_merged11['endtime'] - df_merged11['starttime']).dt.total_seconds() / (60 * 60 * 24)

# View data
df_merged11.head(10)

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,heart_rate,resp_rate,temperature,spo2,glucose,starttime,endtime,hospital_stay_duration,icu_stay_duration,ventilation_duration
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,<NA>,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.807639,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
5,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
6,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
7,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
8,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,<NA>,ELECTIVE,Medicare,...,72.0,14.0,34.700000000,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
9,14985844,23126369,M,71,WHITE,2144-07-19 19:37:00,2144-07-21 15:15:00,<NA>,EW EMER.,Medicare,...,82.0,19.0,36.170000000,99.0,479.0,NaT,NaT,1.818056,1.809329,NaN


In [41]:
# export the data into csv
df_merged11.to_csv('df_merged11.csv', index=False)

In [42]:
from google.colab import files

# Download the CSV file
files.download('df_merged11.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
# read df_merged11.csv
pd.read_csv('df_merged11.csv')

,subject_id,hadm_id,gender,anchor_age,race,admittime,dischtime_x,hospital_expire_flag,admission_type,insurance,...,heart_rate,resp_rate,temperature,spo2,glucose,starttime,endtime,hospital_stay_duration,icu_stay_duration,ventilation_duration
0,19596808,26675460,M,79,WHITE,2128-06-04 16:13:00,2128-06-05 11:36:00,NaN,EW EMER.,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.807639,NaN,NaN
1,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,NaN,ELECTIVE,Medicare,...,72.0,14.0,34.7,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
2,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,NaN,ELECTIVE,Medicare,...,72.0,14.0,34.7,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
3,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,NaN,ELECTIVE,Medicare,...,72.0,14.0,34.7,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
4,15378103,21479747,M,86,WHITE,2183-11-22 13:39:00,2183-12-11 15:59:00,NaN,ELECTIVE,Medicare,...,72.0,14.0,34.7,99.0,126.0,2183-12-06 15:00:00,2183-12-07 09:45:00,19.097222,2.357766,0.78125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36418,12945162,27201085,F,50,WHITE,2169-10-28 12:02:00,2169-10-31 14:18:00,NaN,DIRECT OBSERVATION,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.094444,NaN,NaN
36419,14865704,29199399,F,91,OTHER,2156-11-12 13:59:00,2156-11-13 14:00:00,NaN,AMBULATORY OBSERVATION,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000694,NaN,NaN
36420,14865704,29199399,F,91,OTHER,2156-11-12 13:59:00,2156-11-13 14:00:00,NaN,AMBULATORY OBSERVATION,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000694,NaN,NaN
36421,14865704,29199399,F,91,OTHER,2156-11-12 13:59:00,2156-11-13 14:00:00,NaN,AMBULATORY OBSERVATION,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000694,NaN,NaN
